## Install libraries

In [1]:
# !pip install yahoo_fin

## Import libraries

In [2]:
import all_stocks_symbols as lib_stocks

## Get All Stock Symbols

## Financial: Income, Balance and Cash flow

In [3]:
yearly_json = lib_stocks.get_financials("AAPL",True,False) # yearly
quarterly_json = lib_stocks.get_financials("AAPL",False,True) # quarterly

In [4]:
temp1 = quarterly_json["quarterly_income_statement"]
temp2 = quarterly_json["quarterly_balance_sheet"]
temp3 = quarterly_json["quarterly_cash_flow"]

In [5]:
##add_thousand_comma_separator(temp1,"2021-12-25")

In [6]:
temp1 = lib_stocks.convert_dataframe_reset_index(temp1)
temp11 = lib_stocks.rename_columns_dataframe(temp1)
temp11_1 = lib_stocks.convert_quarters_to_currency_format(temp11)
temp11_1

,Breakdown,2021Q4,2021Q3,2021Q2,2021Q1
0,researchDevelopment,"$6,306,000,000","$5,772,000,000","$5,717,000,000","$5,262,000,000"
1,effectOfAccountingCharges,$0,$0,$0,$0
2,incomeBeforeTax,"$41,241,000,000","$23,248,000,000","$24,369,000,000","$28,011,000,000"
3,minorityInterest,$0,$0,$0,$0
4,netIncome,"$34,630,000,000","$20,551,000,000","$21,744,000,000","$23,630,000,000"
5,sellingGeneralAdministrative,"$6,449,000,000","$5,616,000,000","$5,412,000,000","$5,314,000,000"
6,grossProfit,"$54,243,000,000","$35,174,000,000","$35,255,000,000","$38,079,000,000"
7,ebit,"$41,488,000,000","$23,786,000,000","$24,126,000,000","$27,503,000,000"
8,operatingIncome,"$41,488,000,000","$23,786,000,000","$24,126,000,000","$27,503,000,000"
9,otherOperatingExpenses,$0,$0,$0,$0


In [7]:
temp11.dtypes

Breakdown    object
2021Q4       object
2021Q3       object
2021Q2       object
2021Q1       object
dtype: object